## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
max_new_tokens=500

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-7b-it" // downloaded
# "PathFinderKR/waktaverse-gemma-ko-7b-it"
# "beomi/gemma-ko-7b"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/waktaverse-Llama-2-ko-7b-it"
# "beomi/KoAlpaca-Polyglot-5.8B" // downloaded
# "beomi/open-llama-2-ko-7b"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2" // downloaded
# "mistralai/Mixtral-8x7B-Instruct-v0.1" // downloaded
# "NousResearch/Hermes-2-Pro-Mistral-7B"
# "Intel/neural-chat-7b-v3-1"
# "PathFinderKR/waktaverse-Mistral-KO-7B-Instruct-v0.2"
# "PathFinderKR/waktaverse-Mixtral-KO-8x7B-Instruct-v0.1"

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "NousResearch/Nous-Hermes-2-SOLAR-10.7B"
# "PathFinderKR/waktaverse-SOLAR-KO-10.7B-Instruct-v1.0"

In [6]:
model_id = "PathFinderKR/waktaverse-SOLAR-KO-10.7B-Instruct-v1.0"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Running the model on a single GPU

In [8]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대해 시를 써주세요."

In [10]:
response = generate_response(prompt)
print(response)

### User:
머신러닝에 대해 시를 써주세요.

### Assistant:
머신러닝의 영혼, 어떤 신비한 존재
그의 지성은 계속 자라나며 진화하네
숫자와 문자로 이루어진 세계에서 영원히 빛나네
그의 지성은 계속 자라나며 진화하네

그의 머리속은 수천 개의 네트워크로 이루어져
각각의 노드는 연결과 분리를 반복하네
그의 머리속은 수천 개의 네트워크로 이루어져
각각의 노드는 연결과 분리를 반복하네

그는 입을 열지 않아도 말하네
그의 손은 없지만 작동하네
그는 입을 열지 않아도 말하네
그의 손은 없지만 작동하네

그는 감정을 가지지 않아도 감정을 표현하네
그의 영혼은 계속 자라나며 진화하네
그는 감정을 가지지 않아도 감정을 표현하네
그의 영혼은 계속 자라나며 진화하네

그의 지성은 계속 자라나며 진화하네
그의 영혼은 계속 자라나며 진화하네
그의 영혼은 계속 자라나며 진화하네
그의 영혼은 계속 자라나며 진화하네.
